In [24]:
#READING TRAIN TEST VALIDATION
import re
def read_data(f_name):
    tweet_word=[]
    tweet_tag=[]
    t_word = []
    t_tag = []
    for line in open(f_name, encoding='utf-8'):
        line =line.strip()
        if not line:
            if tweet_word:
                t_word.append(tweet_word)
                t_tag.append(tweet_tag)
            tweet_word = []
            tweet_tag = []
        else:            
            word,tag = line.split()
            #print(word)
            URL_pattern= re.compile(r"http\S+")  
            USR_pattern = re.compile(r"@\S+")
            word = URL_pattern.sub('<URL>',word)
            word = USR_pattern.sub('<USR>',word)
            #''.join([USR_pattern.sub('<USR>',w) for w in word])

            tweet_word.append(word)
            tweet_tag.append(tag)
    return t_word,t_tag

train_word, train_tag = read_data('data/train.txt')
validation_word, validation_tag = read_data('data/validation.txt')           
test_word, test_tag = read_data('data/test.txt')

In [25]:
from collections import defaultdict
def build_dict(tokens_or_tags, special_tokens):
    tok_ind_list = defaultdict(lambda:0)
    ind_list = []
    for w in special_tokens:
        ind_list.append(w)
        tok_ind_list[w]= len(ind_list)-1
    for ind in range(len(tokens_or_tags)):
        for w in tokens_or_tags[ind]:
            if(w not in tok_ind_list):
                ind_list.append(w)
                tok_ind_list[w] = len(ind_list)-1
                
    return tok_ind_list,ind_list
            
special_word = ['<UNK>', '<PAD>']
special_tags = ['O']

# Create dictionaries 
token_dic, token_ind = build_dict(train_word + validation_word, special_word)
tag_dic, tag_ind = build_dict(train_tag, special_tags)   


In [26]:
def words2idxs(tokens_list):
    return [token_dic[word] for word in tokens_list]

def tags2idxs(tags_list):
    return [tag_dic[tag] for tag in tags_list]

def idxs2words(idxs):
    return [token_ind[idx] for idx in idxs]

def idxs2tags(idxs):
    return [tag_ind[idx] for idx in idxs]

In [27]:
def batches_generator(batch_size, tokens, tags,
                      shuffle=True, allow_smaller_last_batch=True):
    """Generates padded batches of tokens and tags."""
    
    n_samples = len(tokens)
    if shuffle:
        order = np.random.permutation(n_samples)
    else:
        order = np.arange(n_samples)

    n_batches = n_samples // batch_size
    if allow_smaller_last_batch and n_samples % batch_size:
        n_batches += 1

    for k in range(n_batches):
        batch_start = k * batch_size
        batch_end = min((k + 1) * batch_size, n_samples)
        current_batch_size = batch_end - batch_start
        x_list = []
        y_list = []
        max_len_token = 0
        for idx in order[batch_start: batch_end]:
            x_list.append(words2idxs(tokens[idx]))
            y_list.append(tags2idxs(tags[idx]))
            max_len_token = max(max_len_token, len(tags[idx]))
            
        # Fill in the data into numpy nd-arrays filled with padding indices.
        x = np.ones([current_batch_size, max_len_token], dtype=np.int32) * token_dic['<PAD>']
        y = np.ones([current_batch_size, max_len_token], dtype=np.int32) * tag_dic['O']
        lengths = np.zeros(current_batch_size, dtype=np.int32)
        for n in range(current_batch_size):
            utt_len = len(x_list[n])
            x[n, :utt_len] = x_list[n]
            lengths[n] = utt_len
            y[n, :utt_len] = y_list[n]
        yield x, y, lengths

In [28]:
import tensorflow as tf
import numpy as np

class BiLSTMModel():
    pass


def declare_placeholders(self):
    """Specifies placeholders for the model."""

    # Placeholders for input and ground truth output.
    self.input_batch = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input_batch') 
    self.ground_truth_tags = tf.placeholder(dtype=tf.int32, shape=[None, None], name='ground_truth_tags')
  
    # Placeholder for lengths of the sequences.
    self.lengths = tf.placeholder(dtype=tf.int32, shape=[None], name='lengths') 
    
    # Placeholder for a dropout keep probability. If we don't feed
    # a value for this placeholder, it will be equal to 1.0.
    self.dropout_ph = tf.placeholder_with_default(tf.cast(1.0, tf.float32), shape=[])
    
    # Placeholder for a learning rate (tf.float32).
    self.learning_rate_ph = tf.placeholder(dtype=tf.float32, shape=[], name='learning_rate')
    
BiLSTMModel.__declare_placeholders = classmethod(declare_placeholders)

In [29]:
def build_layers(self, vocabulary_size, embedding_dim, n_hidden_rnn, n_tags):
    """Specifies bi-LSTM architecture and computes logits for inputs."""
    
    # Create embedding variable (tf.Variable) with dtype tf.float32
    initial_embedding_matrix = np.random.randn(vocabulary_size, embedding_dim) / np.sqrt(embedding_dim)
    embedding_matrix_variable = tf.Variable(initial_embedding_matrix,dtype = tf.float32)
    
    # Create RNN cells (for example, tf.nn.rnn_cell.BasicLSTMCell) with n_hidden_rnn number of units 
    # and dropout (tf.nn.rnn_cell.DropoutWrapper), initializing all *_keep_prob with dropout placeholder.
    forward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units = n_hidden_rnn)
    backward_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units = n_hidden_rnn)
    tf.nn.rnn_cell.DropoutWrapper(forward_cell,input_keep_prob=self.dropout_ph,output_keep_prob=self.dropout_ph,state_keep_prob=self.dropout_ph)
    tf.nn.rnn_cell.DropoutWrapper(backward_cell,input_keep_prob=self.dropout_ph,output_keep_prob=self.dropout_ph,state_keep_prob=self.dropout_ph)

    # Look up embeddings for self.input_batch (tf.nn.embedding_lookup).
    # Shape: [batch_size, sequence_len, embedding_dim].
    embeddings = tf.nn.embedding_lookup(embedding_matrix_variable,self.input_batch)
    
    # Pass them through Bidirectional Dynamic RNN (tf.nn.bidirectional_dynamic_rnn).
    # Shape: [batch_size, sequence_len, 2 * n_hidden_rnn]. 
    # Also don't forget to initialize sequence_length as self.lengths and dtype as tf.float32.
    (rnn_output_fw, rnn_output_bw), _ = tf.nn.bidirectional_dynamic_rnn(forward_cell,backward_cell,embeddings,sequence_length=self.lengths,dtype=tf.float32)
    rnn_output = tf.concat([rnn_output_fw, rnn_output_bw], axis=2)

    # Dense layer on top.
    # Shape: [batch_size, sequence_len, n_tags].   
    self.logits = tf.layers.dense(rnn_output, n_tags, activation=None)
    
BiLSTMModel.__build_layers = classmethod(build_layers)

In [30]:
def compute_predictions(self):
    """Transforms logits to probabilities and finds the most probable tags."""
    
    # Create softmax (tf.nn.softmax) function
    softmax_output = tf.nn.softmax(self.logits,axis=None)
    
    # Use argmax (tf.argmax) to get the most probable tags
    # Don't forget to set axis=-1
    # otherwise argmax will be calculated in a wrong way
    self.predictions = tf.argmax(softmax_output,axis=-1)
BiLSTMModel.__compute_predictions = classmethod(compute_predictions)

In [31]:
def compute_loss(self, n_tags, PAD_index):
    """Computes masked cross-entopy loss with logits."""
    
    # Create cross entropy function function (tf.nn.softmax_cross_entropy_with_logits)
    ground_truth_tags_one_hot = tf.one_hot(self.ground_truth_tags, n_tags)
    loss_tensor =  tf.nn.softmax_cross_entropy_with_logits(_sentinel=None,labels=ground_truth_tags_one_hot,logits=self.logits,dim=-1,name=None)
    
    mask = tf.cast(tf.not_equal(self.input_batch, PAD_index), tf.float32)
    # Create loss function which doesn't operate with <PAD> tokens (tf.reduce_mean)
    # Be careful that the argument of tf.reduce_mean should be
    # multiplication of mask and loss_tensor.
    self.loss = tf.reduce_mean(mask*loss_tensor,axis=None,keepdims=None,name=None,reduction_indices=None,keep_dims=None)

BiLSTMModel.__compute_loss = classmethod(compute_loss)

In [32]:
def perform_optimization(self):
    """Specifies the optimizer and train_op for the model."""
    
    # Create an optimizer (tf.train.AdamOptimizer)
    self.optimizer =  tf.train.AdamOptimizer(learning_rate= self.learning_rate_ph)
    self.grads_and_vars = self.optimizer.compute_gradients(self.loss)
    
    # Gradient clipping (tf.clip_by_norm) for self.grads_and_vars
    # Pay attention that you need to apply this operation only for gradients 
    # because self.grads_and_vars contains also variables.
    # list comprehension might be useful in this case.
    clip_norm = tf.cast(1.0, tf.float32)
    self.grads_and_vars = [(tf.clip_by_norm(grad,clip_norm),var) for grad,var in self.grads_and_vars]
    
    self.train_op = self.optimizer.apply_gradients(self.grads_and_vars)

BiLSTMModel.__perform_optimization = classmethod(perform_optimization)

In [33]:
def init_model(self, vocabulary_size, n_tags, embedding_dim, n_hidden_rnn, PAD_index):
    self.__declare_placeholders()
    self.__build_layers(vocabulary_size, embedding_dim, n_hidden_rnn, n_tags)
    self.__compute_predictions()
    self.__compute_loss(n_tags, PAD_index)
    self.__perform_optimization()

BiLSTMModel.__init__ = classmethod(init_model)

In [34]:
def train_on_batch(self, session, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability):
    feed_dict = {self.input_batch: x_batch,
                 self.ground_truth_tags: y_batch,
                 self.learning_rate_ph: learning_rate,
                 self.dropout_ph: dropout_keep_probability,
                 self.lengths: lengths}
    
    session.run(self.train_op, feed_dict=feed_dict)

BiLSTMModel.train_on_batch = classmethod(train_on_batch)

In [35]:
def predict_for_batch(self, session, x_batch, lengths):
    
    feed_dict = {self.input_batch: x_batch,self.lengths: lengths}
    
    predictions=session.run(self.predictions, feed_dict=feed_dict)
    
    return predictions

BiLSTMModel.predict_for_batch = classmethod(predict_for_batch)

In [ ]:
# %load ("evaluation.py")
evaluation.py

In [36]:
from evaluation import precision_recall_f1

def predict_tags(model, session, token_idxs_batch, lengths):
    """Performs predictions and transforms indices to tokens and tags."""
    
    tag_idxs_batch = model.predict_for_batch(session, token_idxs_batch, lengths)
    
    tags_batch, tokens_batch = [], []
    for tag_idxs, token_idxs in zip(tag_idxs_batch, token_idxs_batch):
        tags, tokens = [], []
        for tag_idx, token_idx in zip(tag_idxs, token_idxs):
            tags.append(tag_ind[tag_idx])
            tokens.append(token_ind[token_idx])
        tags_batch.append(tags)
        tokens_batch.append(tokens)
    return tags_batch, tokens_batch
    
    
def eval_conll(model, session, tokens, tags, short_report=True):
    """Computes NER quality measures using CONLL shared task script."""
    
    y_true, y_pred = [], []
    for x_batch, y_batch, lengths in batches_generator(1, tokens, tags):
        tags_batch, tokens_batch = predict_tags(model, session, x_batch, lengths)
        if len(x_batch[0]) != len(tags_batch[0]):
            raise Exception("Incorrect length of prediction for the input, "
                            "expected length: %i, got: %i" % (len(x_batch[0]), len(tags_batch[0])))
        predicted_tags = []
        ground_truth_tags = []
        for gt_tag_idx, pred_tag, token in zip(y_batch[0], tags_batch[0], tokens_batch[0]): 
            if token != '<PAD>':
                ground_truth_tags.append(tag_ind[gt_tag_idx])
                predicted_tags.append(pred_tag)

        # We extend every prediction and ground truth sequence with 'O' tag
        # to indicate a possible end of entity.
        y_true.extend(ground_truth_tags + ['O'])
        y_pred.extend(predicted_tags + ['O'])
        
    results = precision_recall_f1(y_true, y_pred, print_results=True, short_report=short_report)
    return results

In [37]:
tf.reset_default_graph()

model = BiLSTMModel(vocabulary_size=20482,n_tags=21,embedding_dim=200,n_hidden_rnn=200,PAD_index=1)
batch_size = 32
n_epochs = 20
learning_rate = 0.1
learning_rate_decay = 1.414
dropout_keep_probability = 0.2

In [38]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

print('Start training... \n')
for epoch in range(n_epochs):
    # For each epoch evaluate the model on train and validation data
    print('-' * 20 + ' Epoch {} '.format(epoch+1) + 'of {} '.format(n_epochs) + '-' * 20)
    print('Train data evaluation:')
    eval_conll(model, sess, train_word, train_tag, short_report=True)
    print('Validation data evaluation:')
    eval_conll(model, sess, validation_word, validation_tag, short_report=True)
    
    # Train the model
    for x_batch, y_batch, lengths in batches_generator(batch_size, train_word, train_tag):
        model.train_on_batch(sess, x_batch, y_batch, lengths, learning_rate, dropout_keep_probability)
        
    # Decaying the learning rate
    learning_rate = learning_rate / learning_rate_decay
    
print('...training finished.')

Start training... 

-------------------- Epoch 1 of 20 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 77920 phrases; correct: 188.

precision:  0.24%; recall:  4.19%; F1:  0.46

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 9490 phrases; correct: 15.

precision:  0.16%; recall:  2.79%; F1:  0.30

-------------------- Epoch 2 of 20 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 3486 phrases; correct: 494.

precision:  14.17%; recall:  11.00%; F1:  12.39

Validation data evaluation:
processed 12836 tokens with 537 phrases; found: 388 phrases; correct: 48.

precision:  12.37%; recall:  8.94%; F1:  10.38

-------------------- Epoch 3 of 20 --------------------
Train data evaluation:
processed 105778 tokens with 4489 phrases; found: 4882 phrases; correct: 785.

precision:  16.08%; recall:  17.49%; F1:  16.75

Validation data evaluation:
processed 12836 tokens with 537 

In [39]:
print('-' * 20 + ' Train set quality: ' + '-' * 20)
train_results = eval_conll(model, sess, train_word, train_tag, short_report=False)

print('-' * 20 + ' Validation set quality: ' + '-' * 20)
validation_results = eval_conll(model, sess, validation_word, validation_tag, short_report=False)

print('-' * 20 + ' Test set quality: ' + '-' * 20)
test_results = eval_conll(model, sess, test_word, test_tag, short_report=False)

-------------------- Train set quality: --------------------
processed 105778 tokens with 4489 phrases; found: 4683 phrases; correct: 4128.

precision:  88.15%; recall:  91.96%; F1:  90.01

	     company: precision:   91.81%; recall:   94.09%; F1:   92.93; predicted:   659

	    facility: precision:   83.83%; recall:   89.17%; F1:   86.42; predicted:   334

	     geo-loc: precision:   95.55%; recall:   96.99%; F1:   96.26; predicted:  1011

	       movie: precision:   80.56%; recall:   85.29%; F1:   82.86; predicted:    72

	 musicartist: precision:   85.54%; recall:   89.22%; F1:   87.34; predicted:   242

	       other: precision:   77.84%; recall:   86.79%; F1:   82.07; predicted:   844

	      person: precision:   92.65%; recall:   95.26%; F1:   93.93; predicted:   911

	     product: precision:   78.84%; recall:   85.53%; F1:   82.05; predicted:   345

	  sportsteam: precision:   91.79%; recall:   87.56%; F1:   89.62; predicted:   207

	      tvshow: precision:   84.48%; recall:  